In [55]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [56]:
deliveries = pd.read_csv("C:\\Users\\SHETTS60\\Documents\\Python scripts\\Cricket analytics Materials\\Advanced Cricket Analytics (Masterclass Materials)\\ipl_ball_by_ball_data.csv")
matches = pd.read_csv("C:\\Users\\SHETTS60\\Documents\\Python scripts\\Cricket analytics Materials\\MSD Analysis\\matches.csv")

In [57]:
df = deliveries.copy()
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2008,2008-04-18,M Chinnaswamy Stadium,2,3.1,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,JH Kallis,I Sharma,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2008,2008-04-18,M Chinnaswamy Stadium,2,3.2,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,JH Kallis,I Sharma,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2008,2008-04-18,M Chinnaswamy Stadium,2,3.3,Royal Challengers Bangalore,Kolkata Knight Riders,JH Kallis,W Jaffer,I Sharma,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2008,2008-04-18,M Chinnaswamy Stadium,2,3.4,Royal Challengers Bangalore,Kolkata Knight Riders,JH Kallis,W Jaffer,I Sharma,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [102]:
df["striker"].unique()

array(['SC Ganguly', 'W Jaffer', 'JH Kallis', 'CL White', 'MV Boucher',
       'BB McCullum', 'Mohammad Hafeez', 'R Dravid', 'V Kohli', 'P Kumar',
       'Z Khan', 'SB Joshi', 'AA Noffke', 'B Akhil', 'RT Ponting',
       'DJ Hussey', 'M Kaif', 'RA Jadeja', 'M Rawat', 'D Salunkhe',
       'DS Lehmann', 'SK Warne', 'IK Pathan', 'YK Pathan', 'T Kohli',
       'SM Katich', 'SR Watson', 'G Gambhir', 'SK Trivedi', 'S Dhawan',
       'V Sehwag', 'KC Sangakkara', 'MEK Hussey', 'JDP Oram', 'SK Raina',
       'S Badrinath', 'ML Hayden', 'PA Patel', 'MS Dhoni', 'Yuvraj Singh',
       'JR Hopes', 'K Goel', 'A Symonds', 'AS Yadav', 'SB Styris',
       'RP Singh', 'SB Bangar', 'WPUJC Vaas', 'Y Venugopal Rao',
       'VVS Laxman', 'AC Gilchrist', 'WP Saha', 'RG Sharma', 'LR Shukla',
       'ST Jayasuriya', 'RV Uthappa', 'AM Nayar', 'PR Shah', 'SM Pollock',
       'Harbhajan Singh', 'DJ Thornely', 'L Ronchi', 'LRPL Taylor',
       'S Chanderpaul', 'Kamran Akmal', 'PP Chawla', 'DPMD Jayawardene',
     

In [103]:
batsman_name = "RG Sharma"

In [104]:
batsman_df = df[df["striker"] == batsman_name]

In [105]:
# if value is 'nan' then type will be float. Other than nan type will be str, hence we will consider only value which has str type
def is_bowlers_wicket(player_dismissed, dismissal_kind):
    if type(player_dismissed ) == str:
        if dismissal_kind not in [ 'run out', 'retired hurt', 'obstructing the field']:
            return 1
        else:
            return 0
    else:
        return 0
    

def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1 
    
def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1 

In [106]:
batsman_df['isBowlerWk'] = batsman_df.apply(lambda x:is_bowlers_wicket(x["player_dismissed"],x["wicket_type"]),axis =1)

In [107]:
#combining all into one Function

def ByCustom(df):

    
    df['isDot'] = df['runs_off_bat'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne'] = df['runs_off_bat'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo'] = df['runs_off_bat'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['runs_off_bat'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour'] = df['runs_off_bat'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['runs_off_bat'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(['bowler'])['runs_off_bat'].sum()).rename(columns ={"runs_off_bat":"runs"}).reset_index()
    balls = pd.DataFrame(df.groupby(['bowler'])['match_id'].count()).rename(columns ={"match_id":"balls"}).reset_index()
    innings = pd.DataFrame(df.groupby(['bowler'])['match_id'].apply(lambda x: len(list(np.unique(x))))).rename(columns ={"match_id":"innings"}).reset_index()
    dismissed = pd.DataFrame(df.groupby(['bowler'])['isBowlerWk'].sum()).rename(columns ={"isBowlerWk":"dismissals"}).reset_index()
    
    dots = pd.DataFrame(df.groupby(['bowler'])['isDot'].sum()).rename(columns ={"isDot":"dots"}).reset_index()
    ones = pd.DataFrame(df.groupby(['bowler'])['isOne'].sum()).rename(columns ={"isOne":"ones"}).reset_index()
    twos = pd.DataFrame(df.groupby(['bowler'])['isTwo'].sum()).rename(columns ={"isTwo":"twos"}).reset_index()
    threes = pd.DataFrame(df.groupby(['bowler'])['isThree'].sum()).rename(columns ={"isThree":"threes"}).reset_index()
    fours = pd.DataFrame(df.groupby(['bowler'])['isFour'].sum()).rename(columns ={"isFour":"fours"}).reset_index()
    sixes = pd.DataFrame(df.groupby(['bowler'])['isSix'].sum()).rename(columns ={"isSix":"sixes"}).reset_index()
    
    df = pd.merge(innings,runs, on ='bowler').merge(balls, on ='bowler').merge(dismissed,on ='bowler').merge(dots,on ='bowler').merge(ones,on ='bowler').merge(twos,on ='bowler').merge(threes,on ='bowler').merge(fours,on ='bowler').merge(sixes,on ='bowler')
   
    df['RPI'] = df.apply(lambda x:(x['runs']/x['innings']), axis=1)
    
    #Strike rate
    df['SR'] = df.apply(lambda x:(100 * x['runs']/x['balls']), axis=1)
    #balls per dismissals
    df['BPD'] = df.apply(lambda x:balls_per_dismissal(x['balls'],x['dismissals']), axis=1)
    #balls Per boundary
    df['BPB'] = df.apply(lambda x: balls_per_boundary(x['balls'], (x['fours'] + x['sixes'])), axis = 1)
    #dot Percentage
    df['dot_percentage'] = df.apply(lambda x: x['dots']/x['balls'], axis = 1)
    return df

In [108]:
fav_df = ByCustom(batsman_df)


In [109]:
fav_df.sort_values(["dismissals","SR"], ascending = (False,True )).reset_index(drop = True)

,bowler,innings,runs,balls,dismissals,dots,ones,twos,threes,fours,sixes,RPI,SR,BPD,BPB,dot_percentage
0,A Mishra,17,87,91,7,31,50,4,1,2,3,5.117647,95.604396,13.000000,18.200000,0.340659
1,R Vinay Kumar,6,22,34,6,24,4,3,0,3,0,3.666667,64.705882,5.666667,11.333333,0.705882
2,SP Narine,17,129,120,6,48,51,5,0,14,2,7.588235,107.500000,20.000000,7.500000,0.400000
3,DJ Bravo,15,82,75,5,31,30,3,0,10,1,5.466667,109.333333,15.000000,6.818182,0.413333
4,Sandeep Sharma,11,37,42,4,26,9,2,0,3,2,3.363636,88.095238,10.500000,8.400000,0.619048
5,SK Trivedi,9,45,38,4,16,15,1,0,4,2,5.000000,118.421053,9.500000,6.333333,0.421053
6,UT Yadav,14,143,97,4,41,29,3,0,18,6,10.214286,147.422680,24.250000,4.041667,0.422680
7,DS Kulkarni,9,65,42,4,19,11,1,0,7,4,7.222222,154.761905,10.500000,3.818182,0.452381
8,JA Morkel,7,16,21,3,9,10,1,0,1,0,2.285714,76.190476,7.000000,21.000000,0.428571
9,RA Jadeja,16,71,74,3,21,45,5,0,1,2,4.437500,95.945946,24.666667,24.666667,0.283784


In [110]:
nm_df = ByCustom(batsman_df)

In [111]:
nm_df[nm_df["balls"] > 10].sort_values(["SR"], ascending = (False )).reset_index(drop = True)

,bowler,innings,runs,balls,dismissals,dots,ones,twos,threes,fours,sixes,RPI,SR,BPD,BPB,dot_percentage
0,JDS Neesham,1,28,11,0,2,4,0,0,3,2,28.000000,254.545455,11.000000,2.200000,0.181818
1,MM Patel,3,32,13,0,3,2,2,0,5,1,10.666667,246.153846,13.000000,2.166667,0.230769
2,MF Maharoof,2,32,14,1,3,4,2,0,3,2,16.000000,228.571429,14.000000,2.800000,0.214286
3,DT Christian,5,70,32,0,8,12,1,0,5,6,14.000000,218.750000,32.000000,2.909091,0.250000
4,Mujeeb Ur Rahman,2,26,12,0,4,4,0,0,1,3,13.000000,216.666667,12.000000,3.000000,0.333333
5,VR Aaron,4,59,28,1,9,8,2,0,3,5,14.750000,210.714286,28.000000,3.500000,0.321429
6,Joginder Sharma,3,37,18,0,5,7,0,0,3,3,12.333333,205.555556,18.000000,3.000000,0.277778
7,CJ Anderson,2,44,22,1,8,4,1,0,8,1,22.000000,200.000000,22.000000,2.444444,0.363636
8,JP Faulkner,5,33,17,2,8,3,0,0,3,3,6.600000,194.117647,8.500000,2.833333,0.470588
9,FA Allen,1,25,13,0,2,7,0,0,3,1,25.000000,192.307692,13.000000,3.250000,0.153846
